In [ ]:
import numpy as np
import json
from typing import TypedDict, List, Optional

# LangGraph & LangChain 관련
from langgraph.graph import StateGraph, END

# 임베딩 & 수학 연산 관련
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SLM (Llama.cpp) 관련
from llama_cpp import Llama

# =============================================================================
# [설정 영역] 사용자 환경에 맞게 수정하세요.
# =============================================================================

# 1. 사용할 모델 파일 경로 (다운로드 받은 GGUF 파일 경로)
# 모델이 없다면 테스트를 위해 USE_REAL_SLM = False로 설정하면 가짜 응답으로 흐름 확인 가능
MODEL_PATH = "./models/Qwen2.5-Coder-3B-Instruct-Q4_K_M.gguf" 
USE_REAL_SLM = False  # True: 실제 모델 사용, False: 시뮬레이션 모드

# 2. 타겟 스키마 (우리가 매핑하고자 하는 표준 컬럼들)
TARGET_SCHEMA = [
    "user_id", "user_name", "phone_number", "email_address", 
    "signup_date", "last_login", "is_active", "membership_level",
    "total_purchase_amount", "shipping_address"
]

# =============================================================================
# [초기화] 모델 로딩 (메모리에 올리기)
# =============================================================================

print("⏳ 임베딩 모델 로딩 중... (all-MiniLM-L6-v2)")
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
target_embeddings = embed_model.encode(TARGET_SCHEMA) # 타겟 스키마 미리 벡터화
print("✅ 임베딩 모델 로딩 완료!")

llm = None
if USE_REAL_SLM:
    print(f"⏳ SLM 모델 로딩 중... ({MODEL_PATH})")
    try:
        llm = Llama(
            model_path=MODEL_PATH,
            n_ctx=2048,      # 컨텍스트 윈도우 (입력 길이 제한)
            n_threads=4,     # CPU 코어 수에 맞게 조절
            verbose=False
        )
        print("✅ SLM 모델 로딩 완료!")
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}\n👉 USE_REAL_SLM = False로 전환하여 테스트합니다.")
        USE_REAL_SLM = False

# =============================================================================
# [LangGraph State] 데이터 흐름 정의
# =============================================================================
class MappingState(TypedDict):
    source_col: str          # 입력: 분석할 소스 컬럼명
    sample_value: str        # 입력: 실제 데이터 샘플 (힌트용)
    candidates: List[str]    # 중간값: 임베딩이 찾아낸 후보군 (Top-k)
    final_mapping: str       # 결과: SLM이 선택한 최종 컬럼
    reasoning: str           # 결과: SLM의 선택 이유

# =============================================================================
# [Node 1] Retriever (임베딩 검색기)
# =============================================================================
def retriever_node(state: MappingState):
    source_col = state["source_col"]
    print(f"\n--- [Step 1] Retriever 실행: '{source_col}' 검색 ---")
    
    # 1. 소스 컬럼 벡터화
    source_vec = embed_model.encode([source_col])
    
    # 2. 코사인 유사도 계산
    similarities = cosine_similarity(source_vec, target_embeddings)[0]
    
    # 3. 상위 3개 후보 추출 (Top-3)
    # argsort는 오름차순이므로 뒤집어서 큰 값부터 가져옴
    top_k_indices = np.argsort(similarities)[-3:][::-1]
    candidates = [TARGET_SCHEMA[i] for i in top_k_indices]
    
    print(f"   👉 유사도 기반 후보군: {candidates}")
    return {"candidates": candidates}

# =============================================================================
# [Node 2] Reasoner (SLM 추론기)
# =============================================================================
def slm_reasoning_node(state: MappingState):
    source_col = state["source_col"]
    sample_value = state["sample_value"]
    candidates = state["candidates"]
    
    print(f"\n--- [Step 2] SLM 추론 실행 (Candidate Selection) ---")

    # 1. 프롬프트 작성 (Few-shot & Context 제공)
    prompt = f"""<|im_start|>system
You are a Data Engineer expert in Schema Mapping.
Select the most appropriate Target Column from the Candidates list that matches the Source Column and Sample Value.
<|im_end|>
<|im_start|>user
[Input Data]
- Source Column: "{source_col}"
- Sample Data: "{sample_value}"
- Candidates: {candidates}

[Instruction]
Analyze the meaning of the Source Column and the pattern of the Sample Data.
Then, choose ONE target column from the candidates.
If none match perfectly, choose the closest one.

Output format should be JSON:
{{
    "selected_column": "target_column_name",
    "reason": "short explanation"
}}
<|im_end|>
<|im_start|>assistant
"""
    
    # 2. 모델 추론 (Inference)
    if USE_REAL_SLM and llm:
        # 실제 SLM 호출
        output = llm(
            prompt,
            max_tokens=200,
            stop=["<|im_end|>"],
            temperature=0.1, # 일관된 답변을 위해 낮춤
            echo=False
        )
        response_text = output['choices'][0]['text'].strip()
    else:
        # [시뮬레이션 모드] 모델 파일이 없을 때 로직 확인용 가짜 응답
        print("   ⚠️ (Simulation Mode) SLM 호출을 흉내냅니다.")
        if "hp" in source_col or "010" in sample_value:
            response_text = '{"selected_column": "phone_number", "reason": "Sample data is a Korean mobile number format."}'
        else:
            response_text = f'{{"selected_column": "{candidates[0]}", "reason": "Highest similarity."}}'

    # 3. 결과 파싱 (JSON)
    try:
        # 가끔 모델이 코드 블록(```json ... ```)을 넣을 수 있어 제거 처리
        cleaned_text = response_text.replace("```json", "").replace("```", "").strip()
        result_json = json.loads(cleaned_text)
        
        final_col = result_json.get("selected_column", "Unknown")
        reason = result_json.get("reason", "No reason provided")
        
    except json.JSONDecodeError:
        # 파싱 실패 시 원본 텍스트 반환
        final_col = "Parsing Error"
        reason = response_text

    print(f"   🤖 SLM의 선택: {final_col}")
    print(f"   💡 이유: {reason}")

    return {"final_mapping": final_col, "reasoning": reason}

# =============================================================================
# [Graph Construction] 파이프라인 조립
# =============================================================================
workflow = StateGraph(MappingState)

# 노드 등록
workflow.add_node("retriever", retriever_node)
workflow.add_node("reasoner", slm_reasoning_node)

# 엣지 연결 (순서: Start -> Retriever -> Reasoner -> End)
workflow.set_entry_point("retriever")
workflow.add_edge("retriever", "reasoner")
workflow.add_edge("reasoner", END)

# 컴파일
app = workflow.compile()

# =============================================================================
# [Execution] 테스트 실행
# =============================================================================

if __name__ == "__main__":
    # 테스트 케이스 1: 애매한 약어 컬럼
    input_data = {
        "source_col": "m_hp",         # 단순히 'hp'만 보면 home phone인지 헷갈림
        "sample_value": "010-1234-5678" # 데이터는 명확히 휴대폰 번호
    }

    print("🚀 [Start] 스키마 매핑 에이전트 시작")
    result = app.invoke(input_data)
    
    print("\n============================================")
    print(f"🎯 최종 매핑 결과: [{input_data['source_col']}] -> [{result['final_mapping']}]")
    print(f"📝 추론 근거: {result['reasoning']}")
    print("============================================")